In [1]:
import torch
import torch.nn as nn

In [2]:
from helpers_wo_embedding import CVFConfigDataset

from torch.utils.data import DataLoader, random_split

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
class MLPByHand(nn.Module):
    def __init__(self, N, in_channels, out_channels):
        super().__init__()
        self.linear1 = nn.Linear(in_channels, 32)
        self.linear2 = nn.Linear(32, 32)
        self.out = torch.nn.Linear(32, out_channels, bias=True)

    def forward(self, x, A):
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
        x = torch.relu(x)
        x = self.out(x)
        return x

    def fit(self, train_loader, epochs):
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(
            self.parameters(), lr=0.01, weight_decay=0.01
        )  # weight_decay is a L2 regularization parameter
        for epoch in range(1, epochs + 1):
            self.train()
            total_loss = 0
            count = 0
            for batch in train_loader:
                x = batch[0].to(device)
                y = batch[1].to(device)
                optimizer.zero_grad()
                out = self(x, train_loader.dataset.dataset.A.to(device))
                loss = criterion(out, y)
                total_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Training set | Epoch:", epoch, "Loss:", total_loss / count)

In [5]:
num_nodes = 4       # N
num_features = 4     # D
num_labels = 1       # O
batch_size = 10      # B

# the following needs to be replaced by some node embeddings
# x = torch.randn((batch_size, num_features, num_nodes)) # B x D x N

# A = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]])  # adjacency matrix ( N x N )

model = MLPByHand(N=num_nodes, in_channels=num_features, out_channels=num_labels)
model.to(device)

# out = model(x, A)   # B x D x O

# print("output", out.shape)

MLPByHand(
  (linear1): Linear(in_features=4, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=32, bias=True)
  (out): Linear(in_features=32, out_features=1, bias=True)
)

In [6]:
dataset = CVFConfigDataset(
    "coloring",
    "graph_random_regular_graph_n4_d3_config_rank_dataset.csv",
    "graph_random_regular_graph_n4_d3_A.json",
    num_nodes,
)

In [7]:
train_split = 0.9
test_split = 0.1

train_set, test_set = random_split(
    dataset, [train_split, test_split]
)

train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
test_loader = DataLoader(test_set, batch_size=100, shuffle=False)

In [8]:
model.fit(train_loader, 100)

Training set | Epoch: 1 Loss: tensor(0.8175, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 2 Loss: tensor(0.4496, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 3 Loss: tensor(0.4191, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 4 Loss: tensor(0.4217, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 5 Loss: tensor(0.4516, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 6 Loss: tensor(0.4343, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 7 Loss: tensor(0.4297, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 8 Loss: tensor(0.4229, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 9 Loss: tensor(0.4366, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 10 Loss: tensor(0.4241, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 11 Loss: tensor(0.4866, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 12 Loss: tensor(0.4532, device

In [9]:
import csv

# testing
torch.no_grad()
# torch.set_printoptions(profile="full")

f = open("test_result.csv", "w", newline='')
csv_writer = csv.writer(f)
csv_writer.writerow(["Actual", "Predicted"])

total_matched = 0
criterion = torch.nn.MSELoss()

total_loss = 0
for batch in test_loader:
    x = batch[0].to(device)
    # x = x.repeat(1, 8, 1)
    y = batch[1].to(device)
    out = model(x, test_loader.dataset.dataset.A.to(device))
    csv_writer.writerows(zip(y.detach().cpu().numpy(), out.detach().cpu().numpy()))
    loss = criterion(out, y)
    out = torch.round(out)
    matched = (out == y).sum().item()
    total_matched += matched
    total_loss += loss
    loss.backward()

f.close()
print(
    "Total matched",
    total_matched,
    "out of",
    len(test_set),
    "| ",
    "Loss:",
    total_loss / len(test_loader),
    "| Accuracy",
    round(total_matched / len(test_set) * 100, 4),
    "%",
)

Total matched 15 out of 25 |  Loss: tensor(0.3446, device='cuda:0', grad_fn=<DivBackward0>) | Accuracy 60.0 %
